In [117]:
import os, glob
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [118]:
Root = os.getcwd()
if os.path.exists('Dataset for Model'): pass
else: os.mkdir('Dataset for Model')

if os.path.exists('Dataset By City'):
    files = glob.glob(os.path.join('Dataset By City', '*'))
    for Dataset_By_City in files:
        os.remove(Dataset_By_City)
else:
    os.makedirs("Dataset By City", exist_ok=True) 

Temperature = pd.read_csv(os.path.join('Dataset for Model', 'Temperature.csv'), header = 0)
Humidity = pd.read_csv(os.path.join('Dataset for Model', 'Humidity.csv'), header = 0)
Pressure = pd.read_csv(os.path.join('Dataset for Model', 'Pressure.csv'), header = 0)
Wind_Speed = pd.read_csv(os.path.join('Dataset for Model', 'Wind Speed.csv'), header = 0)
Wind_Direction = pd.read_csv(os.path.join('Dataset for Model', 'Wind Direction.csv'), header = 0)
Weather = pd.read_csv(os.path.join('Dataset for Model', 'Weather.csv'), header = 0)

In [119]:
Weather_Elements = [Temperature, Humidity, Pressure, Wind_Speed, Wind_Direction]
for Element in range(len(Weather_Elements)):
    Weather_Elements_Data = Weather_Elements[Element]
    for Data in range(Weather_Elements_Data.shape[1]):
        if Weather_Elements_Data.iloc[:, Data].isnull().sum() > 0:
            Weather_Elements_Data[Weather_Elements_Data.columns[Data]] = Weather_Elements_Data[Weather_Elements_Data.columns[Data]].fillna(Weather_Elements_Data[Weather_Elements_Data.columns[Data]].mean())       
    Weather_Elements_Data['Datetime'] = pd.to_datetime(Weather_Elements_Data['Datetime'])

Conditions = [
    'sky is clear',                     
    'few clouds',                       
    'scattered clouds',                 
    'broken clouds',                   
    'overcast clouds',              
    'mist',                      
    'fog',                     
    'haze',                 
    'smoke',                   
    'dust',                      
    'volcanic ash',                   
    'drizzle',                         
    'light intensity drizzle',         
    'light intensity drizzle rain',    
    'light rain',                      
    'moderate rain',                    
    'heavy intensity rain',            
    'very heavy rain',                  
    'light rain and snow',              
    'light snow',                  
    'snow',                             
    'heavy snow',                    
    'shower rain',                      
    'heavy intensity shower rain',    
    'proximity shower rain',           
    'light shower sleet',               
    'sleet',                         
    'shower snow',                      
    'heavy shower snow',                
    'thunderstorm with light rain',     
    'thunderstorm with rain',           
    'thunderstorm',                     
    'proximity thunderstorm',           
    'ragged thunderstorm',              
    'thunderstorm with heavy rain',     
]

# enumerate(conditions) sẽ duyệt qua danh sách và trả về cả chỉ số (index) và giá trị (condition). 
# Ví dụ, nó sẽ trả về cặp (0, 'sky is clear'), (1, 'few clouds'), v.v.
Weather_Mapping = {condition: idx + 1 for idx, condition in enumerate(Conditions)}

for Data in range(1, Weather.shape[1]): 
    # Ánh xạ cột thứ hai trong DataFrame
    Weather[Weather.columns[Data]] = Weather[Weather.columns[Data]].map(Weather_Mapping)

    # Điền giá trị NaN bằng giá trị trung bình (hoặc một giá trị cụ thể) trước khi chuyển đổi
    Mean_Value = Weather[Weather.columns[Data]].mean()

    # Nếu mean_value là NaN (có thể do tất cả giá trị trong cột là NaN), bạn có thể gán một giá trị khác
    if pd.isna(Mean_Value):
        Mean_Value = 0  # hoặc một giá trị hợp lý khác

    # Thay thế NaN bằng giá trị trung bình (hoặc giá trị đã chỉ định) và chuyển đổi sang int
    Weather[Weather.columns[Data]] = Weather[Weather.columns[Data]].fillna(Mean_Value).astype(int)
Weather['Datetime'] = pd.to_datetime(Weather['Datetime'])

In [120]:
Citys = Weather.columns.drop('Datetime')
for Num in range(Citys.shape[0]):
    City_Name = Citys[Num]
    Data =  {
    'Datetime': Weather[Weather.columns[0]],
    'Temperature': Temperature[Temperature.columns[Num + 1]],
    'Humidity': Humidity[Humidity.columns[Num + 1]],
    'Pressure': Pressure[Pressure.columns[Num + 1]],
    'Wind Speed': Wind_Speed[Pressure.columns[Num + 1]],
    'Wind Direction': Wind_Direction[Wind_Direction.columns[Num + 1]],
    'Weather': Weather[Weather.columns[Num + 1]]
    }
    City_Data = pd.DataFrame(Data)
    City_Data.to_csv(os.path.join('Dataset By City', City_Name), index = False) 


In [135]:
City_Name = 'Beersheba'
City_Data = pd.read_csv(os.path.join('Dataset By City', City_Name), header = 0)

X = City_Data[['Temperature', 'Humidity', 'Pressure', 'Wind Speed', 'Wind Direction']]
Y = City_Data['Weather']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Linear_Regression_Model = LinearRegression().fit(X_train, Y_train)


In [136]:
print("Hệ số hồi quy (beta):", Linear_Regression_Model.coef_)
print("Hệ số chặn (intercept):", Linear_Regression_Model.intercept_)

Hệ số hồi quy (beta): [ 3.79068123e-02  8.60325679e-02  1.36847336e-02  7.91655122e-01
 -3.39775550e-04]
Hệ số chặn (intercept): -29.27070190687768


In [137]:
# Dự đoán thời tiết trên tập kiểm tra
Predictions = Linear_Regression_Model.predict(X_test)

print("Giá trị dự đoán:", Predictions)
print("Giá trị thực tế:", Y_test.values)

Giá trị dự đoán: [3.17233626 2.48995727 2.49061932 ... 3.66914713 3.75601167 2.18226207]
Giá trị thực tế: [1 4 1 ... 1 1 1]


In [138]:
# Tính MSE
mse = mean_squared_error(Y_test, Predictions)
print("Mean Squared Error (MSE):", mse)

# Tính R-squared
r2 = r2_score(Y_test, Predictions)
print("R-squared:", r2)

Mean Squared Error (MSE): 14.161927479337495
R-squared: 0.1555212004763875


In [125]:
# def Predict_Future_Weather_Conditions(City):